In [1]:
import random
import time
import logging
import mysql.connector
from datetime import datetime
import json
from elasticsearch import Elasticsearch
from datetime import datetime


# Configuración de la base de datos
DB_CONFIG = {
    "host": "localhost",
    "user": "root",
    "password": "miclave",
    "database": "mydatabase",
    "port": 3306
}


In [3]:
# List of the 20 most populated cities in Ecuador

ecuador_cities = [
    "Quito", "Guayaquil", "Cuenca", "SantoDomingo", "Machala", "Manta", "Ambato", "Loja", 
    "Portoviejo", "Esmeraldas", "Riobamba", "Ibarra", "Tulcan", "Quevedo", "Latacunga", "Chone", 
    "Duran", "Loja", "Salinas", "Babahoyo"
]

# Generate the network equipment with corrected city names
network_equipment_corrected = []

# Generate switches (80)
for i in range(1, 81):
    switch_name = f"sw{i}{ecuador_cities[(i - 1) % len(ecuador_cities)]}"
    interfaces = []
    # Add 8 GigaEthernet interfaces with proper sequence
    for j in range(1, 9):
        interfaces.append(f"GigaEthernet{((j - 1) // 8) + 1}/{(j - 1) % 8 + 1}")
    # Add 2 TenGigaEthernet interfaces
    for j in range(1, 3):
        interfaces.append(f"TenGigaEthernet1/{j}")
    network_equipment_corrected.append({"name": switch_name, "interfaces": interfaces})

# Generate routers (20)
for i in range(1, 21):
    router_name = f"ro{i}{ecuador_cities[(i - 1) % len(ecuador_cities)]}"
    interfaces = []
    # Add 2 GigaEthernet interfaces with proper sequence
    for j in range(1, 3):
        interfaces.append(f"GigaEthernet1/{j}")
    # Add 4 TenGigaEthernet interfaces
    for j in range(1, 3):
        interfaces.append(f"TenGigaEthernet1/{j}")
    for j in range(3, 5):
        interfaces.append(f"TenGigaEthernet2/{j}")
    network_equipment_corrected.append({"name": router_name, "interfaces": interfaces})

# Save the corrected data to a JSON file
file_path = './network_equipment_corrected.json'
with open(file_path, 'w') as file:
    json.dump(network_equipment_corrected, file, indent=4)

file_path


'./network_equipment_corrected.json'

In [2]:
es = Elasticsearch(hosts=["http://127.0.0.1:9200"])

## Conexion a Elasticsearch
if es.ping():
    print("Conexión exitosa a Elasticsearch")
else:
    print("Error al conectar con Elasticsearch")
    exit() 


Conexión exitosa a Elasticsearch


C:\Users\equipo\AppData\Local\Temp\ipykernel_24452\505490164.py:4: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  if es.ping():


In [3]:
# Cargar lista de equipos desde un archivo JSON
with open("network_equipment_with_ips.json", "r") as file:
    equipos = json.load(file)

# Lista de equipos caídos
equipos_caidos = []

# Estados de los equipos y sus interfaces
estado_equipos = {
    equipo["name"]: {"disponible": True, "ultimo_down": None, "ultimo_up": datetime.now(), "reiniciar": False} 
    for equipo in equipos
}
estado_interfaces = {
    equipo["name"]: {
        interfaz: {"disponible": True, "ultimo_down": None, "ultimo_up": datetime.now()} 
        for interfaz in equipo["interfaces"]
    }
    for equipo in equipos
}

def generar_log():
    equipo = random.choice(equipos)
    equipo_nombre = equipo["name"]
    equipo_ip = equipo.get("ip", "0.0.0.0")
    ahora = datetime.now()

    # Forzar log en cada iteración si no hay evento relevante
    log = None
    
    # Verificar si el equipo está caído y si han pasado más de 5 minutos
    if not estado_equipos[equipo_nombre]["disponible"]:
        if (ahora - estado_equipos[equipo_nombre]["ultimo_down"]).total_seconds() > 300:
            estado_equipos[equipo_nombre]["disponible"] = True
            estado_equipos[equipo_nombre]["ultimo_up"] = ahora
            log = f"{ahora} availably by ICMP {equipo_nombre} {equipo_ip}"
            if random.random() < 0.5:  # 50% de probabilidad de reinicio justo después
                estado_equipos[equipo_nombre]["reiniciar"] = True
            equipos_caidos.remove(equipo_nombre)  # El equipo vuelve a estar disponible
        return log
    
    if estado_equipos[equipo_nombre]["reiniciar"]:
        estado_equipos[equipo_nombre]["reiniciar"] = False
        return f"{ahora} RESTART {equipo_nombre}"
    
    # Verificar si alguna interfaz está caída y si han pasado más de 5 minutos
    for interfaz, estado in estado_interfaces[equipo_nombre].items():
        if not estado["disponible"] and (ahora - estado["ultimo_down"]).total_seconds() > 300:
            estado["disponible"] = True
            estado["ultimo_up"] = ahora
            return f"{ahora} Interface is up {equipo_nombre} {interfaz}"
    
    # Aumentar la probabilidad de eventos no críticos
    if random.random() < 0.1:  # Aumento la probabilidad a 20% para "Clock Late" o "Interface is clean"
        return f"{ahora} Clock Late {equipo_nombre} {equipo_ip}"

    if random.random() < 0.15:  # Aumento la probabilidad de "Interfaz is clean"
        return f"{ahora} Interface is clean {equipo_nombre}"
    
    # Reducir la probabilidad de eventos críticos
    if random.random() < 0.005:  # 0.5% de probabilidad de caída del equipo
        estado_equipos[equipo_nombre]["disponible"] = False
        estado_equipos[equipo_nombre]["ultimo_down"] = ahora
        equipos_caidos.append(equipo_nombre)  # El equipo se añade a la lista de equipos caídos
        return f"{ahora} unavailably by ICMP {equipo_nombre} {equipo_ip}"
    
    if random.random() < 0.025:  # 2.5% de probabilidad de caída de una interfaz
        interfaz = random.choice(list(estado_interfaces[equipo_nombre].keys()))
        if estado_interfaces[equipo_nombre][interfaz]["disponible"]:
            estado_interfaces[equipo_nombre][interfaz]["disponible"] = False
            estado_interfaces[equipo_nombre][interfaz]["ultimo_down"] = ahora
            return f"{ahora} Interface is down {equipo_nombre} {interfaz}"
    
    # Aumentar la probabilidad de eventos no críticos
    if random.random() < 0.1:  # Aumento la probabilidad a 10%
        return f"{ahora} Debug server log is time"
    
    if random.random() < 0.15:
        return f"{ahora} Network interface port types to discover {equipo_nombre} {random.choice(equipo['interfaces'])}"
    
    if random.random() < 0.15:
        return f"{ahora} availably by SNMP {equipo_nombre} {equipo_ip}"
    
    return log if log else f"{ahora} Debug server log is time"  # Forzamos log si no se generó ninguno

    


In [4]:

def enviar_a_elasticsearch(log):
    # Estructura del log a enviar a Elasticsearch
    doc = {
        "@timestamp": datetime.now(),
        "log": log
    }
    
    # Enviar el log a un índice llamado "network-logs"
    es.index(index="network-logs", document=doc)
    # Ejemplo de uso
    log = generar_log()  # Generamos un log utilizando la función anterior



In [ ]:
import mysql.connector
import re
from datetime import datetime

# Función para enviar el log a MySQL
def enviar_mysql(log):
    # Establecer la conexión con MySQL
    conexion = mysql.connector.connect(host="localhost",user="root",password="miclave",database="template_logs"
    )
    
    # Crear el cursor para ejecutar las consultas
    cursor = conexion.cursor()
    
    # Expresión regular para extraer el timestamp y el mensaje
    pattern = r"^(\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2}\.\d{6})\s+(.*)$"
    match = re.match(pattern, log)
    
    if match:
        timestamp_str = match.group(1)  # El timestamp
        log_message = match.group(2)  # El mensaje del log
        
        # Convertir el timestamp a un objeto datetime
        timestamp = datetime.strptime(timestamp_str, "%Y-%m-%d %H:%M:%S.%f")
        
        # Eliminar los microsegundos (ajustamos el timestamp a solo segundos)
        timestamp = timestamp.replace(microsecond=0)
        
        # Insertar en la tabla 'network_logs'
        sql = "INSERT INTO network_logs (timestamp, log) VALUES (%s, %s)"
        valores = (timestamp, log_message)
        
        # Ejecutar la inserción
        cursor.execute(sql, valores)
        
        # Confirmar la transacción
        conexion.commit()
        
        # Cerrar el cursor y la conexión
        cursor.close()
        conexion.close()

        print(f"Log insertado en MySQL: {timestamp} - {log_message}")
    else:
        print("Formato de log no válido")


In [22]:
try:
    # Generar logs en un loop sin hilos
    while True:
        log = generar_log()
        if log:
            print(log)
            enviar_mysql(log)
            enviar_a_elasticsearch(log)  # Enviar a Elasticsearch
        time.sleep(2)  # Ajustamos el tiempo de impresión a 2 segundos
except KeyboardInterrupt:
        print("\n Inserción de datos del sensor detenida manualmente.")

2025-02-07 23:52:57.250844 Interface is clean sw78Loja


ProgrammingError: 1054 (42S22): Unknown column 'log_message' in 'field list'